In [61]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from Utils import *

import keras.backend as K
K.set_image_data_format('channels_last')

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

In [44]:
import keras.backend as K
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt

In [48]:
def load_dataset():
    train_dataset = h5py.File('Untitled Folder 1/train.h5',"r")
    train_set_x = np.array(train_dataset["train_set_x"[:]])
    train_set_y = np.array(train_dataset["train_set_y"[:]])
    
    test_dataset = h5py.File('Untitled Folder 1/test.h5',"r")
    test_set_x = np.array(test_dataset["test_set_x"[:]])
    test_set_y = np.array(test_dataset["test_set_y"[:]]) #analyzed only basis of happy or angry faces
    
    classes = np.array(test_dataset["list_classes"][:])
    #train_set_x = train_set_x.reshape((1,train_set_x.shape[0]))
    #test_set_x = test_set_x.reshape((1,test_set_x.shape[0]))
    
    return train_set_x, train_set_y,test_set_x,test_set_y,classes

In [49]:
train_set_x, train_set_y, test_set_x, test_set_y, classes = load_dataset()

X_train = train_set_x / 255
X_test = test_set_x / 255

Y_train = train_set_y.reshape((600,1))
Y_test = test_set_y.reshape((150,1))

print("Number of training examples are = "+ str(X_train.shape[0]))
print("Number of testing examples are = "+ str(X_test.shape[0]))
print ("X_train shape" +str(X_train.shape))
print ("X_test shape" +str(X_test.shape))
print ("Y_train shape" +str(Y_train.shape))
print ("Y_test shape" +str(Y_test.shape))

Number of training examples are = 600
Number of testing examples are = 150
X_train shape(600, 64, 64, 3)
X_test shape(150, 64, 64, 3)
Y_train shape(600, 1)
Y_test shape(150, 1)


In [50]:
def model(input_shape):
    X_input =Input(input_shape)
    
    X = ZeroPadding2D((3,3))(X_input)
    
    X = Conv2D(32, (7,7), strides=(1,1), name='conv0')(X)
    X = BatchNormalization(axis=3, name='bn0')(X)
    X = Activation('relu')(X)
    
    X = MaxPooling2D((2,2), name='max_pool')(X)
    
    X = Flatten()(X)
    
    X = Dense(1, activation='sigmoid', name='fc')(X)
    
    model = Model(input= X_input, outputs=X, name='IOTFACE')
    return model

In [51]:
def IOTFACE(input_shape):
    
    X_input = Input(input_shape)
    
    X = ZeroPadding2D((3,3))(X_input)
    
    X = Conv2D(32, (7,7), strides=(1,1), name='conv0')(X)
    X = BatchNormalization(axis=3, name='bn0')(X)
    X = Activation('relu')(X)
    
    X = MaxPooling2D((2,2), name='max_pool')(X)
    
    X = Flatten()(X)
    
    X = Dense(1, activation='sigmoid', name='fc')(X)
    
    model = Model(input= X_input, outputs=X, name='IOTFACE')
    return model
    

In [52]:
iotface = IOTFACE(X_train.shape[1:])

C:\Users\Sukanya1693\Anaconda\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("fc..., name="IOTFACE", inputs=Tensor("in...)`


In [54]:
#compile that model to check accuracy 
iotface.compile (optimizer = 'adam', loss= 'binary_crossentropy', metrics = ["accuracy"])

#train the model
iotface.fit(X_train, Y_train, epochs=40, batch_size= 60)

Epoch 1/40
600/600 [==============================] - 19s 32ms/step - loss: 3.5804 - acc: 0.5283
Epoch 2/40
600/600 [==============================] - 19s 32ms/step - loss: 1.2616 - acc: 0.6650
Epoch 3/40
600/600 [==============================] - 20s 34ms/step - loss: 0.5194 - acc: 0.8033
Epoch 4/40
600/600 [==============================] - 20s 33ms/step - loss: 0.2362 - acc: 0.9000
Epoch 5/40
600/600 [==============================] - 18s 31ms/step - loss: 0.1826 - acc: 0.9200
Epoch 6/40
600/600 [==============================] - 20s 33ms/step - loss: 0.1424 - acc: 0.9483
Epoch 7/40
600/600 [==============================] - 19s 32ms/step - loss: 0.1251 - acc: 0.9650
Epoch 8/40
600/600 [==============================] - 20s 34ms/step - loss: 0.1070 - acc: 0.9667
Epoch 9/40
600/600 [==============================] - 20s 33ms/step - loss: 0.0965 - acc: 0.9767
Epoch 10/40
600/600 [==============================] - 19s 32ms/step - loss: 0.0820 - acc: 0.9850
Epoch 11/40
600/600 [========

In [63]:
preds = iotface.evaluate(X_test, Y_test, batch_size = 32, verbose=1, sample_weight=None)

print()
print("loss = "+str(preds[0]))
print("accuracy ="+str(preds[1]))


150/150 [==============================] - 2s 13ms/step

loss = 0.0827713690201
accuracy =0.953333330949


In [64]:
iotface.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 70, 70, 3)         0         
_________________________________________________________________
conv0 (Conv2D)               (None, 64, 64, 32)        4736      
_________________________________________________________________
bn0 (BatchNormalization)     (None, 64, 64, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pool (MaxPooling2D)      (None, 32, 32, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 32768)             0         
__________

In [75]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files(x86)/Graphviz2.38/bin/'

In [76]:
plot_model(iotface, to_file='iotface.png')

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.